## Prompts for locations

In [1]:
location_prompts = [
    "steampunk city with skyscrapers",
    "cyberpunk village in Japanese rustic style",
    "fantasy dungeons and dragons",
    "noir city from 1930s",
    "StarTrek inspired spaceship",
    "undeground mine of goblins",
    "SuperMario style magic land plain",
    "SuperMario style magic land beach",
]

## Generating locations 

In [2]:
from holodeck import initialize_location
from holodeck import generate_location_and_encounters
import os
from tqdm.notebook import tqdm
import traceback

import concurrent.futures
from tqdm import tqdm

def generate_location(prompt):
    location_dict, encounters_list = generate_location_and_encounters(prompt)
    if location_dict:
        try:
            location = initialize_location(location_dict, encounters_list)
            return location
        except Exception as e:
            print("Error: ", e)
            traceback.print_exc()
    else:
        print(f"GENERATING FROM '{prompt}' failed!")
        return None

locations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(executor.map(generate_location, location_prompts), total=len(location_prompts), desc="Generating locations"))
    locations += [r for r in results if r is not None]



locations

Generating locations:   0%|          | 0/8 [00:00<?, ?it/s]

<
{'name': "Goblin's Mine", 'description': 'A deep underground system of tunnels and ladders, with a few openings visible in the mountainside, leading down to the mine. The entrances are guarded by a group of goblins.\n', 'buildings': [{'name': "Goblin's Entrances", 'description': "The guarded entrances to the Goblin's Mine.", 'enterable': True}], 'ways': [{'name': 'Chasm', 'description': 'A deep chasm that surrounds the mine on all sides. Not even the goblins can cross it.'}]}

<
{'name': 'Starwrecker', 'description': 'A derelict starship, ancient and powerful, nestled among the jagged rocks at the bottom of a deep ravine. Heat and smoke rise from the twisted remains of its engines, and a few fragments remain of its glorious hull. The ship is surrounded by a dense forest of tall trees. \n', 'buildings': [{'name': 'Starwrecker Hull', 'description': 'The few scraps of the hull that remain, strewn among the wreckage.', 'enterable': True}, {'name': 'Broken Engines', 'description': "The tw

Traceback (most recent call last):
  File "/tmp/ipykernel_24014/2627030779.py", line 14, in generate_location
    location = initialize_location(location_dict, encounters_list)
  File "/home/standard/Holodeck/holodeck/game_objects.py", line 184, in initialize_location
    trigger_type = encounter_dict['trigger']['type']
TypeError: list indices must be integers or slices, not str
Generating locations:  12%|█▎        | 1/8 [00:09<01:07,  9.71s/it]

<
[{'probability': 0.1, 'description': 'On the river passage, you encounter a group of mysterious thieves who offer you a chance to join their gang!', 'actions': [{'type': 'character', 'name': 'Mysterious Thieves', 'description': 'A group of unknown thieves who are looking for new members to join their gang.'}], 'trigger': {'type': 'way', 'way': 'River Passage'}}, {'probability': 0.2, 'description': 'As you travel along the main road, you stumble upon a broken steam-powered carriage with a desperate driver.', 'actions': [{'type': 'character', 'name': 'Desperate Driver', 'description': 'A desperate driver who is trying to repair his steam-powered carriage.'}], 'trigger': {'type': 'way', 'way': 'Main Road'}}, {'probability': 0.3, 'description': 'At the power station you find a group of robotic guards patrolling the facility', 'actions': [{'type': 'critter', 'name': 'Robotic Guards'}]}]



Generating locations: 100%|██████████| 8/8 [00:10<00:00,  1.27s/it]

<
[{'probability': 0.2, 'description': 'On the road to the east, you see a shady figure in a hooded cloak.', 'actions': [{'type': 'character', 'name': 'Shady Figure', 'description': 'A hooded figure in a dark cloak, lurking in the shadows of the street lamps.'}], 'trigger': {'type': 'way', 'way': 'Road to East'}}, {'probability': 0.1, 'description': 'As you enter the industrial district, you hear a piercing whistle.', 'actions': [{'type': 'character', 'name': 'Mysterious Whistler', 'description': 'An unseen figure, whistling an eerie tune that echoes through the district.'}], 'trigger': {'type': 'building', 'building': 'Industrial District'}}, {'probability': 0.05, 'description': 'As you approach the port, you see a shadowy figure in the fog.', 'actions': [{'type': 'character', 'name': 'Shadowy Figure'}]}]

<
[{'probability': 0.2, 'description': 'As you enter the cyber village, you hear a strange whirring sound coming from the abandoned factory.', 'actions': [{'type': 'building', 'name

## Generate Image Prompts

In [3]:
from holodeck.gpt_text import \
        generate_object_image_prompt, \
        generate_building_image_prompt, \
        generate_location_image_prompt

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

locations_prompts = []
objects_image_prompts = []
buildings_image_prompts = []

def generate_location_images(location):
    return (location, generate_location_image_prompt(location))

def generate_object_images(location):
    prompts = []
    for o in location.objects:
        prompts.append((o, generate_object_image_prompt(o, location)))
    return prompts

def generate_building_images(location):
    prompts = []
    for b in location.all_buildings:
        prompts.append((b, generate_building_image_prompt(b, location)))
    return prompts

with ThreadPoolExecutor() as executor:
    location_image_futures = list(tqdm(executor.map(generate_location_images, locations), desc="Locations"))
    object_image_futures = [executor.submit(generate_object_images, location) for location in locations]
    building_image_futures = [executor.submit(generate_building_images, location) for location in locations]

    for location, prompt in location_image_futures:
        locations_prompts.append((location, prompt))
    for f in object_image_futures:
        for prompt in f.result():
            objects_image_prompts.append(prompt)
    for f in building_image_futures:
        for prompt in f.result():
            buildings_image_prompts.append(prompt)




Locations: 7it [00:01,  3.80it/s]


## Generate Images

In [4]:
from holodeck.gpt_image import generate_image
from IPython.display import display, Markdown
import PIL.Image as Image

img_prompts = locations_prompts + objects_image_prompts + buildings_image_prompts

# img_prompts = img_prompts[:9]

images = []

for obj, prompt in tqdm(img_prompts, desc="Images"):
    image_bytes = await generate_image(prompt)
    image = Image.open(image_bytes)
    display(Markdown(f"### {obj.name}"))
    display(Markdown(prompt))
    display(image)
    image_file_name = f".images/{obj.name}.jpg"
    image.save(image_file_name)
    image.close()
    image_bytes.close()
    del image_bytes
    images.append((obj, prompt, image_file_name))

len(images)


Images:   0%|          | 0/38 [00:00<?, ?it/s]

### Steampunk City


An aerial photograph of a cityscape with a Power Station, Toll Booth, and Clocktower during nightfall with infrared thermal imaging using an Olympus OM-D E-M5 Mark III — at 16:9. vinkpunk

ValueError: Could not save to PNG for display

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512>

Images:   0%|          | 0/38 [00:10<?, ?it/s]


ValueError: I/O operation on closed file.

## Display Images

In [ ]:
# import textwrap
# import matplotlib.pyplot as plt
# from matplotlib.patheffects import withStroke

# text_width = 30
# dpi = 400.0
# fig_width = 3.234

# columns = 4
# pic_width = fig_width * 0.25


# def plot_images(images):

#     # Estimate the number of rows needed based on the number of images
#     num_rows = (len(images) + columns - 1) // columns

#     # Get the height of the first image adjusted for scale
#     with Image.open(images[0][2]) as img:
#         w, h = img.size
#         aspect_ratio = float(w) / h
#         fig_height = ( h *  ((fig_width * dpi) / w) ) * num_rows

#     fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)

#     # Define the path effect for the outline
#     outline_effect = withStroke(linewidth=0.3, foreground='black')

#     # Loop over the images and create a subplot for each
#     for i, (obj, prompt, image_file_name) in enumerate(images):
#         image = Image.open(image_file_name)
        
#         # Resize the image
#         w, h = image.size
#         aspect_ratio = float(w) / h
#         new_width = int(pic_width * dpi)
#         new_height = int(new_width / aspect_ratio)
#         image = image.resize((new_width, new_height), Resampling.LANCZOS)
        
#         # Create a subplot for the image
#         ax = fig.add_subplot(len(images) // columns + 1, columns, i + 1)
        
#         # Display the image
#         ax.imshow(image)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         ax.set_aspect('equal') # set aspect ratio to 1:1
        
#         # Set the title to the obj.name
#         # ax.set_title(obj.name, fontsize=4, color='magenta', pad=-10)
        
#         # Wrap the prompt text to the desired width
#         wrapped_prompt = textwrap.fill(prompt, width=text_width)
        
#         # Display the wrapped prompt text below the title with black outline
#         ax.text(0.08, -0.3, wrapped_prompt, ha='left', va='center', transform=ax.transAxes, fontsize=2, family='monospace', color='white', path_effects=[outline_effect])
        
#         ax.text(0.5, -0.6, obj.name, ha='center', va='top', transform=ax.transAxes, fontsize=4, family='monospace', color='magenta', path_effects=[outline_effect])

#     fig.subplots_adjust(wspace=0.07, hspace=0.07)

#     plt.tight_layout()

#     # Remove the half-transparent white overlay
#     fig.patch.set_facecolor('none')


# plot_images(images[:4])
